Tehtävä1

In [70]:
def kuntatilastot(url):
    import pandas as pd
    df = pd.read_csv(url,sep=';',decimal=',')
    df = df[(df['maakunta'] == 'Uusimaa') | (df['maakunta'] == 'Päijät-Häme') | (df['maakunta'] == 'Kanta-Häme') | (df['maakunta'] == 'Pirkanmaa')]
    city_df = df[df['kuntamuoto'] == 'Kaupunki'].copy()
    other_df = df[df['kuntamuoto'] == 'Muu kunta'].copy()
    data = {'Tyyppi': ['Kaupunki','Muu kunta'],
            'Keskiarvo': [city_df['Väkiluku'].mean(),other_df['Väkiluku'].mean()],
           'Mediaani': [city_df['Väkiluku'].median(),other_df['Väkiluku'].median()],
           'Summa': [city_df['Väkiluku'].sum(),other_df['Väkiluku'].sum()]}
    
    tulos = pd.DataFrame(data)
    tulos = tulos.round(decimals={'Keskiarvo':0,'Mediaani':0,'Summa':0})
    tulos = tulos.astype({'Keskiarvo':'int32','Mediaani':'int32','Summa':'int32'})
    return tulos
    

kuntatilastot("https://student.labranet.jamk.fi/~pelju/data_av/sources/kunnat.csv")

,Tyyppi,Keskiarvo,Mediaani,Summa
0,Kaupunki,69525,27592,2155287
1,Muu kunta,10887,5403,402825


Tehtävä2

In [111]:
def maakunnat(url):
    import pandas as pd
    df = pd.read_csv(url,sep=';',decimal=',')
    other_df = df[df['kuntamuoto'] == 'Muu kunta']
    result = other_df.groupby(other_df['maakunta']).sum().sort_values('Väkiluku',ascending=False)
    result = result[['Väkiluku','kokonaispinta-ala']]
    tulos = result.head(6)
    print(tulos)
maakunnat("https://student.labranet.jamk.fi/~pelju/data_av/sources/kunnat.csv")

                   Väkiluku  kokonaispinta-ala
maakunta                                      
Uusimaa              219203            5751.35
Pohjois-Pohjanmaa     95571           21035.99
Varsinais-Suomi       82793            7327.68
Pirkanmaa             79856            4493.22
Pohjois-Savo          69339           12357.90
Keski-Suomi           69317           11077.61


Tehtävä3

In [23]:
def kunnat(url):
    import pandas as pd
    df = pd.read_csv(url,sep=';',decimal=',')
    df = df[(df['kuntamuoto'] == 'Kaupunki')].fillna(0)
    df2 = df[(df['Taajama-aste'] > 70) & (df['ev19-KESK'] > df['ev19-PS']) & (df['ev19-KESK'] > df['ev19-KOK']) & (df['ev19-KESK'] > df['ev19-VIHR']) & (df['ev19-KESK'] > df['ev19-RKP']) & (df['ev19-KESK'] > df['ev19-SDP'])].sort_values('ev19-KESK',ascending=False)
    df2['15-64%'] = 100 - (df2['Alle 15-v. osuus%'] + df2['Yli 64-v. osuus%'])
    tulos = df2[['kunta','Taajama-aste','ev19-KESK','15-64%']]
    tulos = tulos.head(10)
    print(tulos)
kunnat("https://student.labranet.jamk.fi/~pelju/data_av/sources/kunnat.csv")

          kunta  Taajama-aste  ev19-KESK  15-64%
75       Kannus          73.2       47.2    58.4
305   Ylivieska          86.0       41.2    59.3
70     Kalajoki          76.0       41.0    56.8
37    Huittinen          70.8       38.2    57.1
275      Tornio          87.8       37.9    60.0
179    Oulainen          74.5       36.3    56.5
87    Kemijärvi          70.1       34.9    52.4
261  Suonenjoki          72.0       31.2    55.4
217       Raahe          87.1       30.7    57.1
194  Pieksämäki          76.1       27.2    57.2


Tehtävä4

In [83]:
def emissions(url):
    import pandas as pd
    df = pd.read_csv(url,sep='\\t',engine='python')
    df.rename(columns={'"country':'country','year"':'year'},inplace=True)
    df['country'] = df['country'].map(lambda x: x.lstrip('"'))
    df['year'] = df['year'].map(lambda x: x.rstrip('"'))
    df = df[(df['main activity sector name'] != '20-99 All stationary installations') & (df['main activity sector name'] != '21-99 All industrial installations (excl. combustion)')]
    tulos = df['main activity sector name'].value_counts().sort_values().copy()
    tulos.rename('sector',inplace=True)
    tulos = tulos.head(5)
    print(tulos)
emissions("https://student.labranet.jamk.fi/~huoptu/datananal_s20/kerta3/harjoitukset/ETS_Database_v38A.csv")

45 Capture of greenhouse gases under Directive 2009/31/EC       14
46 Transport of greenhouse gases under Directive 2009/31/EC     22
40 Production of glyoxal and glyoxylic acid                     37
39 Production of adipic acid                                    43
44 Production of soda ash and sodium bicarbonate               325
Name: sector, dtype: int64


Tehtävä5

In [123]:
def emissions(url):
    import pandas as pd
    df = pd.read_csv(url,sep='\\t',engine='python')
    df.rename(columns={'"country':'country','year"':'year'},inplace=True)
    df['country'] = df['country'].map(lambda x: x.lstrip('"'))
    df['year'] = df['year'].map(lambda x: x.rstrip('"'))
    df = df[['main activity sector name','value','year']].copy()
    df = df[(df['main activity sector name'] != '20-99 All stationary installations') & (df['main activity sector name'] != '21-99 All industrial installations (excl. combustion)')]
    df2 = df[(df['year'] >= '2010') & (df['year'] <= '2015')]
    result = df2.groupby('main activity sector name',as_index=False).sum().sort_values(by='value',ascending=False)
    result = result.iloc[:,[1,0]]
    result.rename(columns=({'main activity sector name':'sector'}),inplace=True)
    tulos = result.head(5)
    print(tulos)
emissions("https://student.labranet.jamk.fi/~huoptu/datananal_s20/kerta3/harjoitukset/ETS_Database_v38A.csv")

           value                               sector
1   1.273006e+10               20 Combustion of fuels
5   2.053915e+09  24  Production of pig iron or steel
10  1.803550e+09      29 Production of cement clinker
2   1.678130e+09          21  Refining of mineral oil
0   5.644274e+08                          10 Aviation
